In [21]:
# import the os module
import os
import shutil
import random

# detect the current working directory and print it
path = os.getcwd()  
print ("The current working directory is %s" % path)  

The current working directory is /home/nina/dataset


## CREATING FILESTRUCTURE


Foldername
<ul>
<ul><li>images</li>
<ul><li>val</li><li>train</li></ul>
<li>labels</li>
<ul><li>val</li><li>train</li></ul>
</ul>

<b>USE</b>: createFilestructure(<i>foldername</i>)

In [28]:
def createDirectory(path):
    try:  
        os.mkdir(path)
    except OSError:  
        print ("Creation of the directory %s failed" % path)
    else:  
        print ("Successfully created the directory %s " % path)
    
            

In [29]:
def createFilestructure(foldername):
    main_dir = ["images", "labels"] 
    sub_dir = ["val", "train"]

    for dir1 in main_dir:
        for dir2 in sub_dir:
            try: os.makedirs(os.path.join(dir1,dir2))
            except OSError: pass
    imgPath = '%s/images'%(path)
    lblPath = '%s/labels'%(path)
    newPath = '%s/%s'%(path, foldername)
    createDirectory(newPath)

    shutil.move(imgPath, newPath) 
    shutil.move(lblPath, newPath) 

createFilestructure("dataN")

### CREATE DATASET


In [30]:
def moveFiles(imageDir, labelDir, outputDir):
    [train, val, test] = splitData(imageDir)
    
    for f in val:
        shutil.copy(('%s/%s.jpg'%(imageDir, f)), ('%s/images/val'%(outputDir)))
        shutil.copy(('%s/%s.txt'%(labelDir, f)), ('%s/labels/val'%(outputDir))) 
    for f in train:
        shutil.copy(('%s/%s.jpg'%(imageDir, f)), ('%s/images/train'%(outputDir)))
        shutil.copy(('%s/%s.txt'%(labelDir, f)), ('%s/labels/train'%(outputDir))) 

In [31]:
def splitData(imageDir):
    files = [f[:-4] for f in os.listdir(imageDir) if f[-4:].lower() == '.jpg'] # getting a list of all imageNames in folder

    # random divide  
    trainval = random.sample(files, int(0.9*len(files)))
    test = [f for f in files if f not in trainval]

    # random divide 
    train = random.sample(trainval, int(0.9*len(trainval)))
    val = [f for f in trainval if f not in train]
    return [train, val, test]

In [34]:
#createFilestructure('editedDataset')

imageDir = '%s/edited'%(path)
labelDir = '%s/edited/labels'%(path)
outputDir = '%s/editedDataset'%(path)
moveFiles(imageDir, labelDir, outputDir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/nina/dataset/edited/labels/9.txt'